In [1]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [(datetime.datetime.fromisoformat(res['time'][:19])).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        
        #temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
                
        #temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def select_data(year_from = 2010, month_from = 1, 
                year_to = 2020, month_to = 8,
                money='USD_JPY', foot = 'H1'):

    filename = str(year_from)+str(month_from)+"to"+str(year_to)+str(month_to)+"_"+money+"_"+foot+ ".csv"
    
    year_months = []

    for i in range(year_from,year_to+1):
        for j in range(month_from,month_to+1):
            temp = [i, j] 
            year_months.append(temp)
    
    
    all_data = []
    for year, month in year_months:
        NY = datetime.timedelta(hours=0)
        date_from = datetime.datetime(year, month, 1) +NY
        date_to = date_from + relativedelta(months=+1, day=1) +NY

        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)


    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    df['Rate']=(df['Close']-df['Open'])/df['Open']
    
    df = df.set_index('Datetime')
    
    df.to_csv(filename, mode = 'w')
    print("'"+filename+"',", end='')
    return df

def main():
    
    #moneys=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY']
    moneys=['GBP_JPY']
    for i in range(len(moneys)):
        select_data(money=moneys[i], foot = 'D')

main()

'20101to20208_GBP_JPY_D.csv',